In [4]:
import numpy as np
import cv2

# Initialisation de la capture vidéo depuis la caméra du PC
cap = cv2.VideoCapture(0)

# Paramètres de l'algorithme
kernel_blur =11
seuil = 40
surface = 1000

# Lecture de la première image pour la référence
ret, originale = cap.read()
if not ret:
    print("Erreur: impossible de capturer l'image initiale.")
    cap.release()
    cv2.destroyAllWindows()
    sys.exit()

originale = cv2.cvtColor(originale, cv2.COLOR_BGR2GRAY)
originale = cv2.GaussianBlur(originale, (kernel_blur, kernel_blur), 0)
kernel_dilate = np.ones((5, 5), np.uint8)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Erreur: impossible de capturer l'image.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (kernel_blur, kernel_blur), 0)
    mask = cv2.absdiff(originale, gray)
    mask = cv2.threshold(mask, seuil, 255, cv2.THRESH_BINARY)[1]
    mask = cv2.dilate(mask, kernel_dilate, iterations=3)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    frame_contour = frame.copy()

    for c in contours:
        cv2.drawContours(frame_contour, [c], 0, (0, 255, 0), 5)
        if cv2.contourArea(c) < surface:
            continue
        x, y, w, h = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # Mise à jour de l'image de référence pour la prochaine itération
    originale = gray

    # Affichage des informations de paramètres sur le cadre
    cv2.putText(frame, f"[o|l]seuil: {seuil}  [p|m]blur: {kernel_blur}  [i|k]surface: {surface}", 
                (10, 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 255, 255), 2)

    # Affichage des fenêtres
    cv2.imshow("frame", frame)
    cv2.imshow("contour", frame_contour)
    cv2.imshow("mask", mask)

    # Gestion des entrées clavier
    key = cv2.waitKey(30) & 0xFF
    if key == ord('q'):
        break
    if key == ord('p'):
        kernel_blur = min(43, kernel_blur + 2)
    if key == ord('m'):
        kernel_blur = max(1, kernel_blur - 2)
    if key == ord('i'):
        surface += 1000
    if key == ord('k'):
        surface = max(1000, surface - 1000)
    if key == ord('o'):
        seuil = min(255, seuil + 1)
    if key == ord('l'):
        seuil = max(1, seuil - 1)

# Libération des ressources
cap.release()
cv2.destroyAllWindows()
